<center><h1><b>PROJECT</b></h1></center>

---

## GUTHENBERG-RICHTER LAW 
(see wikipedia for more)  
In seismology, the Gutenberg–Richter law[1] (GR law) expresses the relationship between the magnitude and total number of earthquakes in any given region and time period of at least that magnitude.
$$ log_{10} ⁡ N = a − b \cdot M $$

where N is the number of events having a magnitude ≥ M, a and b are constants, i.e. they are the same for all values of N and M.
This relationship between event magnitude and frequency of occurrence is remarkably common, although the values of a and b may vary significantly from region to region or over time.   
Some generalization of this formula have been made.

#### PARAMETER b
The parameter b (commonly referred to as the "b-value") is commonly close to 1.0 in seismically active regions. This means that for a given frequency of magnitude 4.0 or larger events there will be 10 times as many magnitude 3.0 or larger quakes and 100 times as many magnitude 2.0 or larger quakes. There is some variation of b-values in the approximate range of 0.5 to 2 depending on the source environment of the region.[5] A notable example of this is during earthquake swarms when b can become as high as 2.5, thus indicating a very high proportion of small earthquakes to large ones. An earthquake swarm is a sequence of seismic events occurring in a local area within a relatively short period. The time span used to define a swarm varies, but may be days, months, or years. Such an energy release is different from the situation when a major earthquake (main shock) is followed by a series of aftershocks: in earthquake swarms, no single earthquake in the sequence is obviously the main shock. In particular, a cluster of aftershocks occurring after a mainshock is not a swarm. The b-value decrease observed prior to the failure of samples deformed in the laboratory[10] has led to the suggestion that this is a precursor to major macro-failure. Alternatively, a b-value significantly different from 1.0 may suggest a problem with the data set; e.g. it is incomplete or contains errors in calculating magnitude. 

#### PARAMETER a
The a-value represents the total seismicity rate of the region. This is more easily seen when the GR law is expressed in terms of the total number of events: 
$$ N = N_{TOT} \cdot 10^{-bM} $$ 
with $N_{TOT} = 10^{a}$ the total number of events (above M=0). Since $10^{a}$ is the total number of events, $10^{-bM}$ must be the probability of those events. 

#### OTHER MODELS
The basic model is the Gutenberg Richter
(GR) model that states the frequency’s logarithm is linearly
dependent on the magnitude. However, Dargahi-Noubary
(1986) and Kagan (1993) suggest that more suitable sta-
tistical models should be used instead of the GR model for
the distribution with high magnitudes. Petersen et al. (2007)
proposed a time-independent model showing that the prob-
ability of earthquake occurrence follows the Poisson dis-
tribution. Considering the time of earthquakes, stochastic
processes, especially Poisson processes, have also been used
to predict the number of earthquakes. However, the Poisson
model cannot be sufficient since it has an exponential recur-
rence time distribution and a constant hazard function. This
assumes that the probability of observing an earthquake at
any given time is independent of both elapsed time since
the last earthquake and its severity. This assumption leads
only to time-independent seismic hazard estimates. Besides,
earthquakes are clustered in time and space and their distri-
bution is over-dispersed compared to the Poisson law. [5]

In recent times, taking into account the seasonality and
trends of earthquakes, the prediction of the size or mag-
nitude has been performed in a few studies by time series
models such as Auto-Regressive Integrated Moving Aver-
age (ARIMA) and generalized autoregressive conditional
heteroskedasticity (GARCH).

a NEW BETTER MODEL IS The SSA is a nonparametric
novel and powerful time series analysis technique incorpo-
rating classical time series analysis, multivariate statistics,
multivariate geometry, dynamical systems, and signal pro-
cessing. This new method can be useful for the prediction of
the earthquake magnitude in a seismic region.

#### ARIMA MODEL
ARIMA, also known as Box-Jenkins models, is divided into
seasonal and non-seasonal models. Non-seasonal Box-Jenkins
models are generally shown as ARIMA (p, d, q), where
p is the parameter of the autoregression (AR) model, d is
the number of difference procedure, and q is the parameter
of the moving average (MA) model. To perform the analysis processes of the Box-Jenkins
method, first, care should be taken to ensure that the series
is free of trend and seasonal fluctuations, that is, the series
should be stationary. Then, determine the p and q param-
eters depending on the autocorrelation function and the par-
tial autocorrelation function graphs and control the model 
parameters’ significance. Lastly, the root mean square error
criterion (RMSE) is used to selects the best model among
the models having the white noise error series

---

## LOADING DATA
We took the data from https://earthquake.usgs.gov/ in two parts because there was a limit in the download size.

In [7]:
df1 = read.table("./data/earthquakes_years_2000_2025.csv", header=TRUE, sep=',')
df2 = read.table("./data/earthquakes_years_1925_2000.csv", header=TRUE, sep=',')
cat("The 1st database has ", nrow(df1), " rows", "\n")
cat("The 2nd database has ", nrow(df2), " rows", "\n")
df = rbind(df1, df2)
cat("The final database has ", nrow(df), " rows", "\n")

The 1st database has  13988  rows 
The 2nd database has  14850  rows 
The final database has  28838  rows 


Let's see the column names:

In [9]:
names(df)

[1] "time"            "latitude"        "longitude"       "depth"          
 [5] "mag"             "magType"         "nst"             "gap"            
 [9] "dmin"            "rms"             "net"             "id"             
[13] "updated"         "place"           "type"            "horizontalError"
[17] "depthError"      "magError"        "magNst"          "status"         
[21] "locationSource"  "magSource"

Meaning of the less obvious comlumns:
* depth: Depth of the earthquake in kilometers (km).
* mag: Magnitude of the earthquake
* magType: Type of magnitude used. (maybe we should confront only earthquake with same magtype?)
* nst: Number of seismic stations that reported the event.
* gap: Largest azimuthal gap in station coverage (degrees).
* dmin: Distance to the nearest station (in degrees).
* rms: Root mean square of the travel time residuals (in seconds).
* net: Network that detected the event (e.g., us, ci, hv). It identifies the organization or seismic network responsible.
* id: The unique identifier for the earthquake event. It’s usually a string combining the net code and a unique event code.
* updated: Time the event information was last updated.
* place: Human-readable location (e.g., "10km SE of Town X").
* type: Type of seismic event (earthquake, quarry blast, etc.). But I only downloaded earthquakes so this column should be useless.
* horizontalError: Horizontal location error (in km or m).
* depthError: Error estimate for the depth (in km).
* magError: Error estimate for the magnitude.
* magNst: Number of stations used for magnitude calculation.
* status: Review status of the event (reviewed, automatic). It means whether the data has been automatically generated or manually reviewed by a seismologist.
* locationSource: Agency that determined the location.
* magSource: Agency that determined the magnitude.